# Segmentación Inteligente de Clientes en Retail Online

**Objetivo**: Segmentar clientes mediante Machine Learning clásico y crear un Producto Mínimo Viable con dashboard para usuarios no técnicos.

**Dataset**: Online Retail - UCI Machine Learning Repository

**Metodología**: 8 pasos secuenciales desde comprensión del problema hasta desarrollo del PMV

---

## PASO 1: Comprensión del Problema

### ¿Qué es un Cliente Valioso en Retail Online?

Un cliente valioso no es solo aquel que realiza una compra grande. En el contexto de retail online, un cliente valioso se caracteriza por:

1. **Lealtad temporal**: Clientes que compran recientemente están más activos y propensos a repetir
2. **Frecuencia de interacción**: Clientes que compran regularmente generan ingresos predecibles y estables
3. **Contribución monetaria**: Clientes que gastan significativamente impactan directamente en los ingresos
4. **Potencial futuro**: Clientes con comportamiento positivo en las tres dimensiones anteriores tienen mayor probabilidad de continuar generando valor

### ¿Por qué No Todos los Clientes Deben Tratarse Igual?

El principio de Pareto se aplica en retail: típicamente el 20% de los clientes genera el 80% de los ingresos. Tratar a todos los clientes de manera uniforme implica:

- **Ineficiencia en recursos**: Invertir lo mismo en clientes de bajo valor que en clientes premium
- **Pérdida de oportunidades**: No identificar clientes de alto valor que necesitan atención especial
- **Riesgo de abandono**: No detectar clientes en riesgo que podrían recuperarse con intervenciones específicas
- **Mal uso del presupuesto**: Gastar en clientes inactivos o perdidos en lugar de en clientes leales

### Decisiones Estratégicas que Apoya la Segmentación

La segmentación de clientes permite tomar decisiones informadas sobre:

1. **Estrategias de retención**: Identificar clientes de alto valor en riesgo para programas de lealtad
2. **Campañas de marketing personalizadas**: Mensajes y ofertas diferenciadas según el segmento
3. **Asignación de presupuesto**: Concentrar inversión en segmentos con mayor ROI
4. **Desarrollo de productos**: Crear ofertas específicas para segmentos con necesidades particulares
5. **Predicción de ingresos**: Entender la composición de la base de clientes para proyecciones financieras
6. **Customer Lifetime Value**: Estimar el valor a largo plazo por segmento para priorizar adquisición

**En resumen**: La segmentación transforma datos transaccionales en inteligencia accionable para optimizar la relación con cada tipo de cliente y maximizar el valor del negocio.

---

## PASO 2: Análisis Exploratorio de Datos (EDA)

En esta fase exploraremos el dataset para entender su estructura, identificar problemas de calidad de datos y obtener insights iniciales.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')  # Usar estilo por defecto
sns.set_theme(style="darkgrid", palette="husl")  # Configuración de seaborn actualizada
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Librerías importadas correctamente")

In [ ]:
# Cargar el dataset
# Nota: Asegúrate de tener el archivo 'Online Retail.xlsx' en la carpeta data/
df = pd.read_excel('../data/Online Retail.xlsx')

print("="*60)
print("CARGA DEL DATASET COMPLETADA")
print("="*60)
print(f"\nDimensiones del dataset: {df.shape[0]:,} filas x {df.shape[1]} columnas")
print("\nPrimeras 5 filas:")
print(df.head())

In [ ]:
# Inspección de la estructura del dataset
print("="*60)
print("ESTRUCTURA DEL DATASET")
print("="*60)
print("\nInformación general:")
print(df.info())

print("\n" + "="*60)
print("TIPOS DE DATOS")
print("="*60)
print(df.dtypes)

In [ ]:
# Identificación de valores faltantes
print("="*60)
print("VALORES FALTANTES")
print("="*60)

missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Columna': missing.index,
    'Valores Faltantes': missing.values,
    'Porcentaje (%)': missing_pct.values
})

print(missing_df)

print("\n" + "="*60)
print("ANÁLISIS DE VALORES FALTANTES EN CustomerID")
print("="*60)
print(f"Total de transacciones sin CustomerID: {df['CustomerID'].isnull().sum():,}")
print(f"Porcentaje: {(df['CustomerID'].isnull().sum() / len(df)) * 100:.2f}%")
print("\nInterpretación: Las transacciones sin CustomerID podrían ser compras de invitados o errores de registro.")

In [ ]:
# Estadística descriptiva
print("="*60)
print("ESTADÍSTICA DESCRIPTIVA")
print("="*60)
print("\nVariables numéricas:")
print(df[['Quantity', 'UnitPrice']].describe())

print("\n" + "="*60)
print("ANÁLISIS DE VARIABLES CATEGÓRICAS")
print("="*60)
print(f"\nNúmero de clientes únicos: {df['CustomerID'].nunique():,}")
print(f"Número de países únicos: {df['Country'].nunique()}")
print(f"Número de productos únicos: {df['StockCode'].nunique():,}")
print(f"Número de facturas únicas: {df['InvoiceNo'].nunique():,}")

print("\n" + "="*60)
print("TOP 5 PAÍSES CON MÁS TRANSACCIONES")
print("="*60)
print(df['Country'].value_counts().head())

In [ ]:
# Visualización de distribuciones
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Distribución de Quantity
axes[0, 0].hist(df['Quantity'], bins=100, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribución de Quantity', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Cantidad')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_xlim(-100, 500)

# Distribución de UnitPrice
axes[0, 1].hist(df['UnitPrice'], bins=100, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title('Distribución de UnitPrice', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Precio Unitario')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_xlim(0, 50)

# Boxplot Quantity
axes[1, 0].boxplot(df['Quantity'].dropna(), vert=True)
axes[1, 0].set_title('Boxplot de Quantity', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Cantidad')

# Boxplot UnitPrice
axes[1, 1].boxplot(df['UnitPrice'].dropna(), vert=True)
axes[1, 1].set_title('Boxplot de UnitPrice', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Precio Unitario')

plt.tight_layout()
plt.show()

print("Visualizaciones generadas correctamente")

In [ ]:
# Detección de outliers y valores anómalos
print("="*60)
print("DETECCIÓN DE OUTLIERS Y VALORES ANÓMALOS")
print("="*60)

# Valores negativos en Quantity
negative_qty = df[df['Quantity'] < 0]
print(f"\nTransacciones con Quantity negativa: {len(negative_qty):,}")
print(f"Porcentaje: {(len(negative_qty) / len(df)) * 100:.2f}%")
print("\nInterpretación: Probablemente sean devoluciones o cancelaciones (InvoiceNo con 'C')")

# Valores negativos o cero en UnitPrice
zero_price = df[df['UnitPrice'] <= 0]
print(f"\nTransacciones con UnitPrice ≤ 0: {len(zero_price):,}")
print(f"Porcentaje: {(len(zero_price) / len(df)) * 100:.2f}%")

# Verificar facturas que inician con 'C' (cancelaciones)
cancelled = df[df['InvoiceNo'].astype(str).str.startswith('C')]
print(f"\nFacturas canceladas (inician con 'C'): {len(cancelled):,}")
print(f"Porcentaje: {(len(cancelled) / len(df)) * 100:.2f}%")

# Estadísticas de outliers extremos
print("\n" + "="*60)
print("ANÁLISIS DE OUTLIERS EXTREMOS")
print("="*60)

Q1_qty = df['Quantity'].quantile(0.25)
Q3_qty = df['Quantity'].quantile(0.75)
IQR_qty = Q3_qty - Q1_qty

Q1_price = df['UnitPrice'].quantile(0.25)
Q3_price = df['UnitPrice'].quantile(0.75)
IQR_price = Q3_price - Q1_price

print(f"\nQuantity - Q1: {Q1_qty:.2f}, Q3: {Q3_qty:.2f}, IQR: {IQR_qty:.2f}")
print(f"UnitPrice - Q1: {Q1_price:.2f}, Q3: {Q3_price:.2f}, IQR: {IQR_price:.2f}")

outliers_qty = df[(df['Quantity'] < Q1_qty - 3*IQR_qty) | (df['Quantity'] > Q3_qty + 3*IQR_qty)]
outliers_price = df[(df['UnitPrice'] < Q1_price - 3*IQR_price) | (df['UnitPrice'] > Q3_price + 3*IQR_price)]

print(f"\nOutliers extremos en Quantity (3*IQR): {len(outliers_qty):,}")
print(f"Outliers extremos en UnitPrice (3*IQR): {len(outliers_price):,}")

### Conclusiones del EDA

**Hallazgos principales:**

1. **Dimensiones**: El dataset contiene más de 540,000 transacciones con 8 columnas
2. **Valores faltantes**: ~25% de transacciones sin CustomerID (compras de invitados o errores)
3. **Valores negativos**: Existen cantidades negativas (devoluciones) y facturas con prefijo 'C' (cancelaciones)
4. **Outliers**: Presencia de outliers extremos en cantidad y precio que deben tratarse
5. **Geografía**: Reino Unido concentra la mayoría de transacciones
6. **Calidad de datos**: Necesaria limpieza antes de modelar

**Acciones necesarias:**
- Eliminar transacciones sin CustomerID (no podemos segmentar sin identificación)
- Eliminar transacciones canceladas y con valores negativos
- Filtrar outliers extremos que puedan distorsionar el análisis
- Calcular valor monetario de cada transacción

---

## PASO 3: Limpieza y Agregación de Datos

Ahora procedemos a limpiar el dataset y agregarlo a nivel de cliente para poder calcular las métricas RFM.

In [ ]:
# Limpieza del dataset
print("="*60)
print("LIMPIEZA DEL DATASET")
print("="*60)

# Crear una copia para trabajar
df_clean = df.copy()

print(f"\nRegistros iniciales: {len(df_clean):,}")

# 1. Eliminar transacciones sin CustomerID
df_clean = df_clean[df_clean['CustomerID'].notna()]
print(f"Después de eliminar CustomerID nulos: {len(df_clean):,}")

# 2. Eliminar transacciones canceladas (InvoiceNo con 'C')
df_clean = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('C')]
print(f"Después de eliminar cancelaciones: {len(df_clean):,}")

# 3. Eliminar cantidad y precios negativos o cero
df_clean = df_clean[df_clean['Quantity'] > 0]
df_clean = df_clean[df_clean['UnitPrice'] > 0]
print(f"Después de eliminar valores negativos/cero: {len(df_clean):,}")

# 4. Convertir InvoiceDate a datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# 5. Calcular el valor monetario de cada transacción
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']

print("\n" + "="*60)
print("DATASET LIMPIO - PRIMERAS FILAS")
print("="*60)
print(df_clean.head())

print(f"\n✓ Limpieza completada: {len(df_clean):,} transacciones válidas")
print(f"✓ Registros eliminados: {len(df) - len(df_clean):,} ({((len(df) - len(df_clean)) / len(df)) * 100:.2f}%)")

In [ ]:
# Agregación de datos a nivel de cliente
print("="*60)
print("AGREGACIÓN A NIVEL DE CLIENTE")
print("="*60)

# Agrupar por CustomerID y calcular métricas básicas
customer_data = df_clean.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',        # Número de facturas únicas (compras)
    'TotalAmount': 'sum',          # Gasto total
    'InvoiceDate': 'max'           # Fecha de última compra
}).reset_index()

# Renombrar columnas
customer_data.columns = ['CustomerID', 'NumPurchases', 'TotalSpent', 'LastPurchaseDate']

print(f"\nNúmero de clientes únicos: {len(customer_data):,}")
print("\nPrimeras 10 filas del dataset agregado:")
print(customer_data.head(10))

print("\n" + "="*60)
print("ESTADÍSTICAS DE CLIENTES")
print("="*60)
print(customer_data[['NumPurchases', 'TotalSpent']].describe())

print(f"\n✓ Agregación completada: {len(customer_data):,} clientes")

## PASO 4: Ingeniería de Variables - Modelo RFM

El modelo RFM es una técnica clásica de segmentación que evalúa a los clientes en tres dimensiones:

- **Recency (R)**: ¿Qué tan recientemente compró el cliente? Clientes que compraron recientemente son más propensos a responder a ofertas y tienen mayor engagement.

- **Frequency (F)**: ¿Con qué frecuencia compra el cliente? Clientes frecuentes son más leales y tienen mayor probabilidad de repetir compras.

- **Monetary (M)**: ¿Cuánto gasta el cliente? Clientes que gastan más tienen mayor valor para el negocio y mayor impacto en ingresos.

**Estas tres métricas juntas nos permiten identificar diferentes tipos de clientes y priorizar estrategias de negocio.**

In [ ]:
# Calcular RFM
print("="*60)
print("CÁLCULO DE MÉTRICAS RFM")
print("="*60)

# Definir la fecha de referencia (un día después de la última compra en el dataset)
reference_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"\nFecha de referencia: {reference_date}")

# Calcular Recency: días desde la última compra
customer_data['Recency'] = (reference_date - customer_data['LastPurchaseDate']).dt.days

# Frequency: ya lo tenemos como NumPurchases
customer_data['Frequency'] = customer_data['NumPurchases']

# Monetary: ya lo tenemos como TotalSpent
customer_data['Monetary'] = customer_data['TotalSpent']

# Crear DataFrame RFM
rfm = customer_data[['CustomerID', 'Recency', 'Frequency', 'Monetary']].copy()

print("\n" + "="*60)
print("DATASET RFM - PRIMERAS 10 FILAS")
print("="*60)
print(rfm.head(10))

print("\n" + "="*60)
print("ESTADÍSTICAS RFM")
print("="*60)
print(rfm[['Recency', 'Frequency', 'Monetary']].describe())

print("\n" + "="*60)
print("INTERPRETACIÓN DE LAS MÉTRICAS")
print("="*60)
print("✓ Recency: Valores BAJOS indican clientes más recientes (mejor)")
print("✓ Frequency: Valores ALTOS indican clientes más frecuentes (mejor)")
print("✓ Monetary: Valores ALTOS indican clientes que gastan más (mejor)")

# Visualizar distribuciones RFM
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(rfm['Recency'], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].set_title('Distribución de Recency', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Días desde última compra')
axes[0].set_ylabel('Frecuencia')

axes[1].hist(rfm['Frequency'], bins=50, edgecolor='black', alpha=0.7, color='lightcoral')
axes[1].set_title('Distribución de Frequency', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Número de compras')
axes[1].set_ylabel('Frecuencia')

axes[2].hist(rfm['Monetary'], bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
axes[2].set_title('Distribución de Monetary', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Gasto total')
axes[2].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

print("\n✓ Métricas RFM calculadas correctamente")

## PASO 5: Segmentación No Supervisada con K-Means

Aplicaremos K-Means sobre las métricas RFM normalizadas para identificar grupos naturales de clientes con comportamientos similares.

In [ ]:
# Normalizar las variables RFM
print("="*60)
print("NORMALIZACIÓN DE VARIABLES RFM")
print("="*60)

# Crear escalador
scaler = StandardScaler()

# Normalizar
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# Crear DataFrame con datos normalizados
rfm_normalized = pd.DataFrame(
    rfm_scaled, 
    columns=['Recency_scaled', 'Frequency_scaled', 'Monetary_scaled']
)

print("\nDatos normalizados - Primeras 5 filas:")
print(rfm_normalized.head())

print("\n" + "="*60)
print("ESTADÍSTICAS DESPUÉS DE NORMALIZACIÓN")
print("="*60)
print(rfm_normalized.describe())

print("\n✓ Normalización completada (media=0, desviación estándar=1)")

In [ ]:
# Probar diferentes valores de K usando el método del codo
print("="*60)
print("DETERMINACIÓN DEL NÚMERO ÓPTIMO DE CLUSTERS")
print("="*60)

# Probar K de 2 a 10
K_range = range(2, 11)
inertias = []
silhouette_scores = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_normalized)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(rfm_normalized, kmeans.labels_))

# Visualizar método del codo y silhouette scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Método del codo
axes[0].plot(K_range, inertias, marker='o', linewidth=2, markersize=8)
axes[0].set_xlabel('Número de Clusters (K)', fontsize=11)
axes[0].set_ylabel('Inercia (Within-cluster sum of squares)', fontsize=11)
axes[0].set_title('Método del Codo', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Silhouette score
axes[1].plot(K_range, silhouette_scores, marker='o', linewidth=2, markersize=8, color='orange')
axes[1].set_xlabel('Número de Clusters (K)', fontsize=11)
axes[1].set_ylabel('Silhouette Score', fontsize=11)
axes[1].set_title('Silhouette Score por K', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Mostrar tabla de resultados
results_df = pd.DataFrame({
    'K': list(K_range),
    'Inercia': inertias,
    'Silhouette Score': silhouette_scores
})

print("\nResultados de evaluación:")
print(results_df)

print("\n" + "="*60)
print("INTERPRETACIÓN")
print("="*60)
print("• Método del Codo: Buscar el punto donde la inercia deja de disminuir significativamente")
print("• Silhouette Score: Valores más altos indican mejor separación entre clusters")
print("• Balance: Priorizar interpretabilidad de negocio sobre optimización extrema")

In [ ]:
# Aplicar K-Means con el número óptimo de clusters
print("="*60)
print("APLICACIÓN DE K-MEANS")
print("="*60)

# Seleccionar K=4 basado en el balance entre métricas e interpretabilidad
optimal_k = 4
print(f"\nNúmero de clusters seleccionado: {optimal_k}")
print("Justificación: K=4 ofrece un buen balance entre separación de clusters")
print("e interpretabilidad para segmentación de clientes en retail.")

# Aplicar K-Means
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
rfm['Cluster'] = kmeans_final.fit_predict(rfm_normalized)

print(f"\n✓ K-Means aplicado correctamente")
print(f"✓ Inercia final: {kmeans_final.inertia_:.2f}")
print(f"✓ Silhouette Score: {silhouette_score(rfm_normalized, rfm['Cluster']):.3f}")

# Distribución de clientes por cluster
print("\n" + "="*60)
print("DISTRIBUCIÓN DE CLIENTES POR CLUSTER")
print("="*60)
cluster_counts = rfm['Cluster'].value_counts().sort_index()
print(cluster_counts)

# Visualización
plt.figure(figsize=(10, 6))
cluster_counts.plot(kind='bar', color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A'], edgecolor='black')
plt.title('Distribución de Clientes por Cluster', fontsize=14, fontweight='bold')
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Número de Clientes', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n✓ Total de clientes segmentados: {len(rfm):,}")

## PASO 6: Interpretación de Segmentos

Ahora analizaremos las características de cada cluster para entender qué tipo de clientes representa y asignar etiquetas descriptivas.

In [ ]:
# Análisis de características de cada cluster
print("="*60)
print("CARACTERÍSTICAS PROMEDIO POR CLUSTER")
print("="*60)

cluster_summary = rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()
print("\nPromedios:")
print(cluster_summary)

print("\n" + "="*60)
print("ESTADÍSTICAS DETALLADAS POR CLUSTER")
print("="*60)

for cluster_id in range(optimal_k):
    cluster_data = rfm[rfm['Cluster'] == cluster_id]
    print(f"\n{'='*60}")
    print(f"CLUSTER {cluster_id}")
    print(f"{'='*60}")
    print(f"Número de clientes: {len(cluster_data):,}")
    print(f"Porcentaje del total: {(len(cluster_data) / len(rfm)) * 100:.2f}%")
    print(f"\nRecency  - Promedio: {cluster_data['Recency'].mean():.1f} días")
    print(f"Frequency - Promedio: {cluster_data['Frequency'].mean():.1f} compras")
    print(f"Monetary - Promedio: £{cluster_data['Monetary'].mean():,.2f}")
    print(f"Monetary - Total: £{cluster_data['Monetary'].sum():,.2f}")

# Calcular contribución de ingresos por cluster
print("\n" + "="*60)
print("CONTRIBUCIÓN A INGRESOS POR CLUSTER")
print("="*60)

revenue_by_cluster = rfm.groupby('Cluster')['Monetary'].sum().sort_values(ascending=False)
revenue_pct = (revenue_by_cluster / rfm['Monetary'].sum()) * 100

revenue_df = pd.DataFrame({
    'Ingreso Total': revenue_by_cluster,
    'Porcentaje (%)': revenue_pct
})

print(revenue_df)

In [ ]:
# Visualización de clusters en espacio RFM
fig = plt.figure(figsize=(15, 5))

# Recency vs Monetary
ax1 = fig.add_subplot(131)
for cluster_id in range(optimal_k):
    cluster_data = rfm[rfm['Cluster'] == cluster_id]
    ax1.scatter(cluster_data['Recency'], cluster_data['Monetary'], 
                label=f'Cluster {cluster_id}', alpha=0.6, s=30)
ax1.set_xlabel('Recency (días)', fontsize=11)
ax1.set_ylabel('Monetary (£)', fontsize=11)
ax1.set_title('Recency vs Monetary', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Frequency vs Monetary
ax2 = fig.add_subplot(132)
for cluster_id in range(optimal_k):
    cluster_data = rfm[rfm['Cluster'] == cluster_id]
    ax2.scatter(cluster_data['Frequency'], cluster_data['Monetary'], 
                label=f'Cluster {cluster_id}', alpha=0.6, s=30)
ax2.set_xlabel('Frequency (compras)', fontsize=11)
ax2.set_ylabel('Monetary (£)', fontsize=11)
ax2.set_title('Frequency vs Monetary', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Recency vs Frequency
ax3 = fig.add_subplot(133)
for cluster_id in range(optimal_k):
    cluster_data = rfm[rfm['Cluster'] == cluster_id]
    ax3.scatter(cluster_data['Recency'], cluster_data['Frequency'], 
                label=f'Cluster {cluster_id}', alpha=0.6, s=30)
ax3.set_xlabel('Recency (días)', fontsize=11)
ax3.set_ylabel('Frequency (compras)', fontsize=11)
ax3.set_title('Recency vs Frequency', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Visualizaciones generadas correctamente")

In [ ]:
# Asignar etiquetas descriptivas a los clusters
print("="*60)
print("ASIGNACIÓN DE ETIQUETAS A LOS SEGMENTOS")
print("="*60)

# Analizar los promedios para asignar nombres
cluster_avg = rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()

# Función para asignar etiquetas basadas en características RFM
def assign_segment_name(cluster_id, cluster_avg):
    """
    Asigna nombres descriptivos basados en las características RFM del cluster
    """
    recency = cluster_avg.loc[cluster_id, 'Recency']
    frequency = cluster_avg.loc[cluster_id, 'Frequency']
    monetary = cluster_avg.loc[cluster_id, 'Monetary']
    
    # Lógica de asignación basada en patrones típicos
    if recency < 50 and frequency > 5 and monetary > 2000:
        return 'Champions'  # Mejores clientes
    elif recency < 100 and frequency > 3 and monetary > 1000:
        return 'Loyal Customers'  # Clientes leales
    elif recency > 200 and frequency < 3:
        return 'At Risk'  # En riesgo de abandono
    else:
        return 'Occasional Buyers'  # Compradores ocasionales

# Crear mapeo de clusters a nombres
segment_names = {}
for cluster_id in range(optimal_k):
    segment_names[cluster_id] = assign_segment_name(cluster_id, cluster_avg)

# Aplicar etiquetas
rfm['Segment'] = rfm['Cluster'].map(segment_names)

print("\nMapeo de Clusters a Segmentos:")
for cluster_id, name in segment_names.items():
    count = len(rfm[rfm['Cluster'] == cluster_id])
    print(f"  Cluster {cluster_id} → {name} ({count:,} clientes)")

print("\n" + "="*60)
print("DESCRIPCIÓN DE CADA SEGMENTO")
print("="*60)

for cluster_id in range(optimal_k):
    segment_name = segment_names[cluster_id]
    cluster_data = rfm[rfm['Cluster'] == cluster_id]
    
    print(f"\n📊 {segment_name.upper()}")
    print(f"{'─'*60}")
    print(f"Características principales:")
    print(f"  • Recency promedio: {cluster_data['Recency'].mean():.0f} días")
    print(f"  • Frequency promedio: {cluster_data['Frequency'].mean():.1f} compras")
    print(f"  • Monetary promedio: £{cluster_data['Monetary'].mean():,.2f}")
    print(f"  • Tamaño: {len(cluster_data):,} clientes ({(len(cluster_data)/len(rfm)*100):.1f}%)")
    print(f"  • Contribución a ingresos: £{cluster_data['Monetary'].sum():,.2f}")
    
    # Importancia para el negocio
    if segment_name == 'Champions':
        print(f"\n  💎 Importancia: CRÍTICA - Son los mejores clientes")
        print(f"     Estrategia: Programas VIP, early access, atención premium")
    elif segment_name == 'Loyal Customers':
        print(f"\n  ⭐ Importancia: ALTA - Base sólida del negocio")
        print(f"     Estrategia: Programas de lealtad, ofertas exclusivas")
    elif segment_name == 'At Risk':
        print(f"\n  ⚠️  Importancia: URGENTE - Requieren retención")
        print(f"     Estrategia: Campañas de reactivación, descuentos personalizados")
    else:
        print(f"\n  📈 Importancia: MEDIA - Potencial de crecimiento")
        print(f"     Estrategia: Incrementar frecuencia con incentivos")

print(f"\n{'='*60}")
print("✓ Segmentos etiquetados e interpretados correctamente")

## PASO 7: Árbol de Decisión Explicativo

Utilizaremos un árbol de decisión NO para predecir, sino para **explicar** las reglas que definen cada segmento de manera interpretable.

In [ ]:
# Entrenar árbol de decisión para explicar los segmentos
print("="*60)
print("ÁRBOL DE DECISIÓN EXPLICATIVO")
print("="*60)

# Preparar datos
X = rfm[['Recency', 'Frequency', 'Monetary']]
y = rfm['Cluster']

# Entrenar árbol con profundidad limitada para interpretabilidad
tree_model = DecisionTreeClassifier(
    max_depth=4,  # Limitar profundidad para mayor interpretabilidad
    min_samples_split=100,
    min_samples_leaf=50,
    random_state=42
)

tree_model.fit(X, y)

print(f"✓ Árbol de decisión entrenado")
print(f"✓ Profundidad del árbol: {tree_model.get_depth()}")
print(f"✓ Número de hojas: {tree_model.get_n_leaves()}")
print(f"✓ Accuracy en datos de entrenamiento: {tree_model.score(X, y):.3f}")

print("\n" + "="*60)
print("IMPORTANCIA DE VARIABLES")
print("="*60)

feature_importance = pd.DataFrame({
    'Variable': ['Recency', 'Frequency', 'Monetary'],
    'Importancia': tree_model.feature_importances_
}).sort_values('Importancia', ascending=False)

print(feature_importance)

# Visualizar el árbol
plt.figure(figsize=(20, 10))
plot_tree(
    tree_model, 
    feature_names=['Recency', 'Frequency', 'Monetary'],
    class_names=[segment_names[i] for i in range(optimal_k)],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title('Árbol de Decisión - Reglas de Segmentación', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("INTERPRETACIÓN DEL ÁRBOL")
print("="*60)
print("El árbol muestra las reglas de decisión que definen cada segmento:")
print("• Cada nodo representa una condición sobre R, F o M")
print("• Las ramas muestran el flujo de decisión")
print("• Las hojas indican el segmento final asignado")
print("\nEsto permite a usuarios no técnicos entender:")
print("- ¿Qué hace que un cliente sea 'Champion'?")
print("- ¿Qué condiciones llevan a un cliente a estar 'At Risk'?")
print("- ¿Cómo se diferencian los segmentos en términos simples?")

print("\n✓ Modelo explicativo completado")

## PASO 8: Preparación para el PMV

Guardamos los datos procesados y el modelo para utilizarlos en el dashboard de Streamlit.

In [ ]:
# Guardar datos procesados para el dashboard
import pickle

print("="*60)
print("GUARDANDO DATOS PARA EL PMV")
print("="*60)

# Guardar el DataFrame RFM con segmentos
rfm.to_csv('../data/rfm_segments.csv', index=False)
print("✓ RFM con segmentos guardado: data/rfm_segments.csv")

# Guardar el mapeo de nombres de segmentos
with open('../data/segment_names.pkl', 'wb') as f:
    pickle.dump(segment_names, f)
print("✓ Nombres de segmentos guardados: data/segment_names.pkl")

# Guardar el escalador
with open('../data/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✓ Escalador guardado: data/scaler.pkl")

# Guardar el modelo de clustering
with open('../data/kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans_final, f)
print("✓ Modelo K-Means guardado: data/kmeans_model.pkl")

print("\n" + "="*60)
print("PREPARACIÓN COMPLETADA")
print("="*60)
print("Todos los archivos necesarios han sido guardados.")
print("El dashboard de Streamlit puede ahora cargar estos datos.")

print("\nPara ejecutar el dashboard:")
print("  streamlit run src/app_dashboard.py")

---

## Conclusiones y Recomendaciones Estratégicas

### Resumen del Análisis

Hemos completado exitosamente la segmentación de clientes mediante Machine Learning clásico:

1. ✅ **Análisis de más de 400,000 transacciones** de ~4,300 clientes únicos
2. ✅ **Segmentación en 4 grupos** con características diferenciadas
3. ✅ **Modelo RFM** como base de la segmentación
4. ✅ **K-Means clustering** para identificación automática de patrones
5. ✅ **Árbol de decisión** para explicar reglas de asignación

### Segmentos Identificados

Los 4 segmentos representan diferentes tipos de comportamiento y valor para el negocio:

| Segmento | Características | Prioridad |
|----------|----------------|-----------|
| **Champions** | Alta frecuencia, alto gasto, reciente | CRÍTICA |
| **Loyal Customers** | Frecuencia media-alta, gasto medio, activos | ALTA |
| **Occasional Buyers** | Baja frecuencia, gasto bajo | MEDIA |
| **At Risk** | Inactivos, riesgo de abandono | URGENTE |

### Recomendaciones Estratégicas

**Para Champions:**
- Programas VIP exclusivos
- Early access a nuevos productos
- Atención personalizada
- Incentivos por referidos

**Para Loyal Customers:**
- Programas de puntos y recompensas
- Ofertas especiales periódicas
- Comunicación frecuente de valor

**Para Occasional Buyers:**
- Campañas para aumentar frecuencia
- Ofertas por volumen
- Recordatorios personalizados

**Para At Risk:**
- **URGENTE**: Campañas de reactivación
- Descuentos significativos
- Encuestas de satisfacción
- Win-back campaigns

### Impacto en el Negocio

- **Personalización**: Mensajes y ofertas adaptados a cada segmento
- **Eficiencia**: Inversión de marketing enfocada en alto ROI
- **Retención**: Identificación temprana de clientes en riesgo
- **Crecimiento**: Estrategias para mover clientes a segmentos superiores

### Próximos Pasos

1. Implementar el dashboard para monitoreo continuo
2. Automatizar la actualización mensual de segmentos
3. Medir impacto de campañas diferenciadas por segmento
4. Refinar segmentación con datos adicionales (canal, categoría de producto)

---

**Análisis completado correctamente. Dashboard listo para implementación.**